https://docs.api.lens.org/request.html

Get token: https://www.lens.org/lens/user/subscriptions

In [165]:
import getpass

In [167]:
token=getpass.getpass('token:')

token:········


In [168]:
size=1000

In [169]:
import requests
url = 'https://api.lens.org/scholarly/search'

data='''{
 "query": {
   "bool": {
     "must": [{
       "match_phrase":{
                "author.affiliation.name": "University of Antioquia"
           }
       },
       {"match": {"year_published": "2003"} }
     ]
   }
 }
}'''

In [170]:
data='''{
 "query": {
   "bool": {
     "must": [{
       "match_phrase":{
                "author.affiliation.name": "University of Antioquia"
           }
       },
       {
       "range": {
       "year_published": {
                "gte": "2003",
                "lte": "2004"
            }
       } 
       }
     ]
   }
 },
 "size": %i
}''' %size

data = '''{
     "query": {
     "bool": {
     "must": [{
           "match_phrase":{
                "author.affiliation.name": "University of Antioquia"
           }        
     },
      {
         "range": {
            "year_published": {
                "gte": "2003",
                "lte": "2004"
            }
        }
   }
   ]}},      
     "size": %i,
     "sort": [
           {
                "year_published": "desc"
           }
     ]
}''' %size

In [171]:
headers = {'Authorization': '{}'.format(token), 'Content-Type': 'application/json'}
response = requests.post(url, data=data, headers=headers)
if response.status_code != requests.codes.ok:
  print(response.status_code)
else:
  r=response.text

In [158]:
import pandas as pd

In [159]:
rr=pd.read_json(r)

In [160]:
rr.shape

(823, 3)

In [161]:
udea=pd.DataFrame( list( rr.data.values ) )

In [162]:
udea.authors

0      [{'affiliations': [{'name': 'University of Ant...
1      [{'affiliations': [{'name': 'University of Ant...
2      [{'affiliations': [{'name': 'University of Ant...
3      [{'affiliations': [{'name': 'University of Ant...
4      [{'affiliations': [{'name': 'University of Ant...
5      [{'affiliations': [{'name': 'University of Ant...
6      [{'affiliations': [{'name': 'University of Ant...
7      [{'affiliations': [{'name': 'University of Ant...
8      [{'affiliations': [{'name': 'University of Ant...
9      [{'affiliations': [{'name': 'University of Ant...
10     [{'affiliations': [{'name': 'University of Ant...
11     [{'affiliations': [{'name': 'School of Public ...
12     [{'affiliations': [{'name': 'University of Ant...
13     [{'affiliations': [{'name': 'University of Ant...
14     [{'affiliations': [{'name': 'Georgia Regents U...
15     [{'affiliations': [{'name': 'Grupo Malaria, Un...
16     [{'affiliations': [{'name': 'University of Ant...
17     [{'affiliations': [{'nam

In [163]:
udea.to_json('data/udea.json.gz',compression='gzip')

In [164]:
pd.read_json('data/udea.json.gz',compression='gzip').reset_index(drop=True)

,abstract,author_count,authors,chemicals,conference,created,date_published,end_page,external_ids,fields_of_study,...,references,references_count,scholarly_citations,scholarly_citations_count,source,source_urls,start_page,title,volume,year_published
0,During the execution of an epidemiological stu...,4,"[{'first_name': 'Marta', 'collective_name': No...",None,None,2018-05-12T08:25:54.018000+00:00,2003-09-01T00:00:00+00:00,526,"[{'type': 'magid', 'value': '2155587119'}, {'t...","[Fauna, Ecology, Lutzomyia, Phlebotominae, Psy...",...,None,NaN,"[002-102-565-260-417, 003-836-515-913-554, 016...",6.0,"{'asjc_subjects': ['Insect Science'], 'asjc_co...","[{'type': 'html', 'url': 'http://www.scielo.br...",523,Phlebotominae fauna (Diptera: Psychodidae) in ...,32,2003
1,Abstract We present an experimental and theore...,3,"[{'first_name': 'Alejandro', 'collective_name'...",None,None,2018-05-12T18:30:13.363000+00:00,2003-01-01T00:00:00+00:00,39,"[{'type': 'magid', 'value': '2088927943'}, {'t...","[Binding site, Mole, Adsorption, Molecular mod...",...,"[{'lens_id': '001-121-008-373-096'}, {'lens_id...",45.0,"[004-280-757-816-373, 004-950-504-222-761, 007...",72.0,"{'asjc_subjects': ['General Chemistry'], 'asjc...","[{'type': 'unknown', 'url': 'http://www.scienc...",29,CO2 adsorption on carbonaceous surfaces: a com...,41,2003
2,Abstract The objective was to determine whethe...,5,"[{'first_name': 'Mario F', 'collective_name': ...",None,None,2018-05-11T17:48:10.693000+00:00,2004-08-01T00:00:00+00:00,2458,"[{'type': 'magid', 'value': '2164019474'}, {'t...","[Gene–environment interaction, Animal science,...",...,"[{'lens_id': '004-498-199-431-656'}, {'lens_id...",16.0,"[005-890-293-742-170, 011-165-466-150-890, 016...",13.0,{'asjc_subjects': ['Animal Science and Zoology...,"[{'type': 'unknown', 'url': 'http://www.scienc...",2455,Genotype x environment interaction for age at ...,87,2004
3,The novel incomplete cuboidal cluster [W3Se4H3...,8,"[{'first_name': 'Manuel G.', 'collective_name'...","[{'mesh_id': 'D000143', 'substance_name': 'Aci...",None,2018-05-12T02:43:05.861000+00:00,2004-01-19T00:00:00+00:00,536,"[{'type': 'magid', 'value': '2076435597'}, {'t...","[Crystal structure, Cluster (physics), Ion, Ha...",...,"[{'lens_id': '002-328-335-396-820'}, {'lens_id...",27.0,"[003-451-445-376-601, 003-536-992-350-868, 004...",21.0,{'title': 'Dalton Transactions'},"[{'type': 'html', 'url': 'https://pubs.rsc.org...",530,Synthesis and structure of the incomplete cubo...,None,2004
4,We suggest an elementary scheme for introducin...,2,"[{'first_name': 'L', 'collective_name': None, ...",None,None,2018-05-12T19:02:05.920000+00:00,2003-03-07T00:00:00+00:00,259,"[{'type': 'magid', 'value': '1975553101'}, {'t...","[Particle, Physics, Quadratic equation, Anharm...",...,[{'lens_id': '160-665-526-243-743'}],1.0,"[060-385-023-288-497, 061-055-647-292-704, 160...",3.0,{'asjc_subjects': ['General Physics and Astron...,"[{'type': 'unknown', 'url': 'http://iopscience...",253,The bouncing bob: quasi-classical states,24,2003
5,Si bien el ajuste del sistema contable publico...,5,"[{'first_name': 'John Cardona', 'collective_na...",None,None,2018-05-14T17:23:01.636000+00:00,2004-01-01T00:00:00+00:00,200,"[{'type': 'magid', 'value': '577037698'}]","[Performance art, History]",...,None,NaN,None,NaN,None,"[{'type': 'html', 'url': 'https://aprendeenlin...",171,APUNTES PARA LA EVALUACIÓN Y REVISIÓN DEL MARC...,None,2004
6,None,1,"[{'first_name': 'Augusto Escobar', 'collective...",None,None,2018-05-14T23:05:03.013000+00:00,2003-01-01T00:00:00+00:00,122,"[{'type': 'magid', 'value': '1563293002'}]",None,...,None,NaN,None,NaN,None,"[{'type': 'html', 'url': 'https://biblat.unam....",105,Diálogo con Javier Echeverri:una aventura de v...,None,2003
7,Resumen Con el objetivo de detectar y aislar e...,3,"[{'first_name': 'Silvia', 'collective_name': N...",None,None,2018-05-15T22:34:18.530000+00:00,2003-01-01T00:00:00+00:00,45,"[{'type': 'magid', 'value': '2170850714'}]","[Surgery, Virology,